In [53]:
!pip install tika
!pip install nlp
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.7 MB/s eta 0:00:00


In [54]:
#import regular expressions
import re 
from tika import parser
import spacy
from spacy.matcher import Matcher
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
file = r'Pratham Shankwalker - Resume.pdf'
file_data = parser.from_file(file)
text = file_data['content']
print(type(text))

<class 'str'>


In [56]:
# declare a dictionary to store all the extracted text
extracted_text = {}

In [57]:
#function to extract email adress
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)
  
email = get_email_addresses(text)
print(email)

extracted_text["E-Mail"] = email[0]
extracted_text

['pshankwalker1@gmail.com', 'pshankwalker1@gmail.com']


{'E-Mail': 'pshankwalker1@gmail.com'}

In [58]:
#function to get phone number
def get_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

phone_number= get_phone_numbers(text)
print(phone_number)

#add phone number to the extracted text 
extracted_text["Phone Number"] = phone_number

['9307051204']


In [59]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

name = extract_name(text)
print(name)


Github |


In [60]:
def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    skills = ["machine learning",
             "deep learning",
             "nlp",
             "natural language processing",
             "mysql",
             "sql",
             "django",
             "computer vision",
              "tensorflow",
             "opencv",
             "mongodb",
             "artificial intelligence",
             "ai",
             "flask",
             "robotics",
             "data structures",
             "python",
             "c++",
             "matlab",
             "css",
             "html",
             "github",
             "php",
             "Neural Networks",
              "statistics",
              "Transfer Learning"]
            
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


skills = []
skills = extract_skills(text)
extracted_text['skills']=skills
skills

['Github',
 'Machine learning',
 'Opencv',
 'Natural language processing',
 'Tensorflow',
 'Data structures',
 'Python',
 'Mysql',
 'C++',
 'Sql']

In [61]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'B TECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

education = extract_education(text)
education
extracted_text["Qualification"] = education

In [62]:
extracted_text

{'E-Mail': 'pshankwalker1@gmail.com',
 'Phone Number': ['9307051204'],
 'skills': ['Github',
  'Machine learning',
  'Opencv',
  'Natural language processing',
  'Tensorflow',
  'Data structures',
  'Python',
  'Mysql',
  'C++',
  'Sql'],
 'Qualification': [('BE', '2024')]}

In [63]:
#college/institution
sub_patterns = ['[A-Z][a-z]* College of Engineering','[A-Z][a-z]* Educational Institute',
                'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]*']
pattern = '({})'.format('|'.join(sub_patterns))
matches = re.findall(pattern, text)

extracted_text["Institute"] = matches
matches

['Goa College of Engineering']

In [64]:
#experience

sub_patterns = ['[A-Z][a-z]* [A-Z][a-z]* Private Limited','[A-Z][a-z]* [A-Z][a-z]* Pvt. Ltd.','[A-Z][a-z]* [A-Z][a-z]* Inc.', '[A-Z][a-z]* LLC',
                ]
pattern = '({})'.format('|'.join(sub_patterns))
Exp = re.findall(pattern, text)
extracted_text["Experience"] = Exp
Exp

[]

In [65]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [66]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(skills)

In [68]:
stopw  = set(stopwords.words('english'))
df =pd.read_csv('job_final.csv') 

In [70]:
df['test']=df['Job_Description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))
df['test']

0       About company: Smart Food Safe Solutions Inc. ...
1       Location Bangalore Experience Years Job Descri...
2       Open Systems International, Inc. (OSI) www.osi...
3       About Job Software Testing Engineer Job Descri...
4       Location: Bangalore Experience: 6Years Skills ...
                              ...                        
1919    Skills Qualifications: Years experience Strong...
1920    Job TH10519_13189 Posted on: 29th May, 2019Job...
1921    Job Description spend percent lives buildings....
1922    (Job Number: 1905027) Job Title â€“ Web Develo...
1923    marry design engineering language ways produce...
Name: test, Length: 1924, dtype: object

In [71]:
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
test = (df['test'].values.astype('U'))

In [72]:
def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

In [73]:
distances, indices = getNearestN(test)
test = list(test) 
matches = []


In [74]:
for i,j in enumerate(indices):
    dist=round(distances[i][0],2)
  
    temp = [dist]
    matches.append(temp)
    
matches = pd.DataFrame(matches, columns=['Match confidence'])

In [78]:
df['match']=matches['Match confidence']
df1=df.sort_values('match')
df1[['Position', 'Company','Location','url']].head(10).reset_index()

,index,Position,Company,Location,url
0,777,Machine Learning Engineer,Unbxd Inc,â€“ Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
1,32,Machine Learning Engineer,Involvio,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
2,1378,Data Engineer (Funding Circle),Bloom Consulting Services,Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
3,1237,Data Engineer (Spark & Spark),Accion Labs,Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
4,33,Machine Learning Engineers,Convate Consultancy Services Private Limited,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
5,377,Machine Learning-Consultant,Wipro LTD,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
6,27,Machine learning Engineer,Thoucentric,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
7,509,Machine Learning Developer,Pattem Digital,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
8,38,Machine Learning-Lead,Wipro LTD,– Bengaluru,https://www.glassdoor.co.in/partner/jobListing...
9,144,Data Science Engineer,Eminence Core Solutions LLP,– Pune,https://www.glassdoor.co.in/partner/jobListing...
